In [10]:
pip install fastapi jinja2 modal multion notion-client sqlalchemy openai pydantic

  Obtaining dependency information for multion from https://files.pythonhosted.org/packages/c1/01/e350dd5f12174c81b9b57dd3ee9bdb0d5125cf9332b9319596d792198140/multion-1.3.6-py3-none-any.whl.metadata
  Obtaining dependency information for notion-client from https://files.pythonhosted.org/packages/22/5e/977505b17c5528cffaaf561de46f58fa546345ae75866e3985039aaa991d/notion_client-2.2.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/7c/ae/2a66f072b9733b3287807f76bca53913aa448c7ac1d70b08cd9e79af7675/openai-1.40.3-py3-none-any.whl.metadata
  Obtaining dependency information for agentops<0.3.0,>=0.2.3 from https://files.pythonhosted.org/packages/5a/f2/f90538b00d887c04a5570e8a3af4aef27a600a67c058a0ee6befafd60748/agentops-0.2.6-py3-none-any.whl.metadata
  Obtaining dependency information for httpx>=0.21.2 from https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.

In [8]:
conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from typing import List

from fastapi import FastAPI, HTTPException, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from modal import Image, App, Secret, asgi_app, mount
from multion.client import MultiOn

import templates
from db import get_notion_crm_api_key, get_notion_database_id, store_notion_crm_api_key, store_notion_database_id, \
    clean_all_transcripts_except, append_segment_to_transcript, remove_transcript
from llm import news_checker
from models import Memory
from notion_utils import store_memoy_in_db

app = FastAPI()

modal_app = App(
    name='plugins_examples',
    secrets=[Secret.from_dotenv('.env')],
    mounts=[
        mount.Mount.from_local_dir('templates/', remote_path='templates/'),
    ]
)

multion = MultiOn(api_key=os.getenv('MULTION_API_KEY', '123'))


@modal_app.function(
    image=Image.debian_slim().pip_install_from_requirements('requirements.txt'),
    keep_warm=1,  # need 7 for 1rps
    memory=(1024, 2048),
    cpu=4,
    allow_concurrent_inputs=10,
)
@asgi_app()
def plugins_app():
    return app


def call_multion(books: List[str]):
    print('Buying books with MultiOn')
    response = multion.browse(
        cmd=f"Add to my cart the following books (in paperback version, or any physical version): {books}",
        url="https://amazon.com",
        local=True,
    )
    return response.message


# **************************************************
# ************ On Memory Created Plugin ************
# **************************************************

# noinspection PyRedeclaration
templates = Jinja2Templates(directory="templates")


@app.get('/setup-notion-crm', response_class=HTMLResponse)
async def setup_notion_crm(request: Request, uid: str):
    if not uid:
        raise HTTPException(status_code=400, detail='UID is required')
    return templates.TemplateResponse("setup_notion_crm.html", {"request": request, "uid": uid})


@app.post('/creds/notion-crm', response_class=HTMLResponse)
def creds_notion_crm(request: Request, uid: str = Form(...), api_key: str = Form(...), database_id: str = Form(...)):
    if not api_key or not database_id:
        raise HTTPException(status_code=400, detail='API Key and Database ID are required')
    print({'uid': uid, 'api_key': api_key, 'database_id': database_id})
    store_notion_crm_api_key(uid, api_key)
    store_notion_database_id(uid, database_id)
    return templates.TemplateResponse("okpage.html", {"request": request, "uid": uid})


@app.get('/setup/notion-crm')
def is_setup_completed(uid: str):
    notion_api_key = get_notion_crm_api_key(uid)
    notion_database_id = get_notion_database_id(uid)
    return {'is_setup_completed': notion_api_key is not None and notion_database_id is not None}


@app.post('/notion-crm')
def notion_crm(memory: Memory, uid: str):
    print(memory.dict())
    notion_api_key = get_notion_crm_api_key(uid)
    if not notion_api_key:
        return {'message': 'Your Notion CRM plugin is not setup properly. Check your plugin settings.'}

    store_memoy_in_db(notion_api_key, get_notion_database_id(uid), memory)
    return {}


# *******************************************************
# ************ On Transcript Received Plugin ************
# *******************************************************


@app.post('/news-checker')
def news_checker_endpoint(uid: str, data: dict):
    session_id = data['session_id']  # use session id in case your plugin needs the whole conversation context
    new_segments = data['segments']
    clean_all_transcripts_except(uid, session_id)

    transcript: list[dict] = append_segment_to_transcript(uid, session_id, new_segments)
    message = news_checker(transcript)

    if message:
        # so that in the next call with already triggered stuff, it doesn't trigger again
        remove_transcript(uid, session_id)

    return {'message': message}

# https://e604-107-3-134-29.ngrok-free.app/news-checker

ModuleNotFoundError: No module named 'db'